In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import env
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
##def acquire_data():
 #   df = pd.read_csv('anonymized-curriculum-access.txt', sep= ' ', names=['date', 'time', 'web_link', 'number', 'number_two', 'ip' ])
 #   df['timestamp'] = df['date'] + ' '+ df['time']
 #   df.drop(columns=['date', 'time'], inplace= True)
 #   new_order = ['ip', 'number', 'number_two',  'timestamp', 'web_link']
#
 #   # use the reindex method to reorder the columns
 #   df = df.reindex(columns=new_order)
 #   df['timestamp'] = pd.to_datetime(df['timestamp'])
 #   df = df.set_index('timestamp').sort_index()
 #   return df

In [125]:
# Import .txt file and convert it to a DataFrame object
#df = pd.read_table("anonymized-curriculum-access.txt", sep = '\s', header = None, 
#                   names = ['date', 'time', 'page', 'id', 'cohort', 'ip'])

C:\Users\juang\AppData\Local\Temp\ipykernel_24240\3331995557.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_table("anonymized-curriculum-access.txt", sep = '\s', header = None,


In [3]:
def get_connection(db, user=env.user, host=env.host, password=env.password):
    '''
    This function gets conncection to mySQL database
    '''
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'


def new_data():
    '''
    This function reads the curriculum data from the mySQL database into a df.
    '''
    # Create SQL query.
    sql_query = '''
    SELECT 
        date,
        time,
        ip,
        path as page,
        user_id as id,
        cohort_id as cohort,
        name as cohort_name,
        slack,
        start_date,
        end_date,
        program_id
    FROM
        curriculum_logs.logs
    join
        curriculum_logs.cohorts on cohort_id = id
    ;
    '''
    # Read in DataFrame from Codeup db.
    df = pd.read_sql(sql_query, get_connection('curriculum_logs'))

    return df

def get_data():
    '''
    This function reads in curriculum data from curriculum_log database, writes data to
    a csv file if a local file does not exist, and returns a df.
    '''
    if os.path.isfile('curriculum.csv'):

        # If csv file exists, read in data from csv file.
        df = pd.read_csv('curriculum.csv', index_col=0)

    else:

        # Read fresh data from db into a DataFrame.
        df = new_data()

        # Write DataFrame to a csv file.
        df.to_csv('curriculum.csv')

    return df

In [4]:
def prep_data(df):
    '''
    This function takes in a messy dataframe and return the cleaned verison of dataframe.
    Detial steps are in code comment below.
    '''
    # Change date columns to datetime
    df.date = pd.to_datetime(df.date)
    df.start_date = pd.to_datetime(df.start_date)
    df.end_date = pd.to_datetime(df.end_date)
    # Set date columns as datatime as index
    df = df.set_index(df.date)

    # Drop null values
    df = df.dropna()

    # Encode program_id
    df['program'] = df.program_id.map({1: 'Full Stack PHP', 2: 'Full Stack Java', 3: 'Data Science', 4: 'Front End'})

    return df

In [5]:
df = new_data()

In [7]:
df = prep_data(df)

In [8]:
df

,date,time,ip,page,id,cohort,cohort_name,slack,start_date,end_date,program_id,program
date,,,,,,,,,,,,
2018-01-26,2018-01-26,09:55:03,97.105.19.61,/,1,8.0,Hampton,#hampton,2015-09-22,2016-02-06,1,Full Stack PHP
2018-01-26,2018-01-26,09:56:02,97.105.19.61,java-ii,1,8.0,Hampton,#hampton,2015-09-22,2016-02-06,1,Full Stack PHP
2018-01-26,2018-01-26,09:56:05,97.105.19.61,java-ii/object-oriented-programming,1,8.0,Hampton,#hampton,2015-09-22,2016-02-06,1,Full Stack PHP
2018-01-26,2018-01-26,09:56:06,97.105.19.61,slides/object_oriented_programming,1,8.0,Hampton,#hampton,2015-09-22,2016-02-06,1,Full Stack PHP
2018-01-26,2018-01-26,09:56:24,97.105.19.61,javascript-i/conditionals,2,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2,Full Stack Java
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-21,2021-04-21,16:41:51,71.150.217.33,jquery/personal-site,64,28.0,Staff,#,2014-02-04,2014-02-04,2,Full Stack Java
2021-04-21,2021-04-21,16:42:02,71.150.217.33,jquery/mapbox-api,64,28.0,Staff,#,2014-02-04,2014-02-04,2,Full Stack Java
2021-04-21,2021-04-21,16:42:09,71.150.217.33,jquery/ajax/weather-map,64,28.0,Staff,#,2014-02-04,2014-02-04,2,Full Stack Java


Email to analyst:


Hello,


I have some questions for you that I need to be answered before the board meeting Thursday afternoon. I need to be able to speak to the following questions. I also need a single slide that I can incorporate into my existing presentation (Google Slides) that summarizes the most important points. My questions are listed below; however, if you discover anything else important that I didn’t think to ask, please include that as well.


1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?
2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?
3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?
4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?
5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?
6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?
7. Which lessons are least accessed?
8. Anything else I should be aware of?


Thank you,


___________________


Other info:


• To get 100 on this project you only need to answer 5 out of the 7 questions
• send your email before the due date and time to staff-oneil@codeup.com with the subject line Curriculum Anomaly Analysis - [Name]
• Submit a link to a final notebook on GitHub that asks and answers questions - document the work you do to justify findings
• Compose an email with the answers to the questions/your findings, and in the email, include the link to your notebook in GitHub
• You will not present this, so be sure that the details you need your leader to convey/understand are clearly communicated in the email.
• Continue to use best practices of acquire.py, prepare.py, etc.
• Since there is no modeling to be done for this project, there is no need to split the data into train/validate/test
• The cohort schedule is in the SQL database, and alumni.codeup.com has info as well.

In [17]:
first_access = df.groupby('id').date.min()

In [18]:
#create df from first access date
first_access_date = pd.DataFrame({'first_access_date': first_access}).reset_index()

In [19]:
#add cohort info to id
first_access_date['cohort'] = df.groupby('id').cohort.unique()

In [20]:
first_access_date.isna().sum()

id                    0
first_access_date     0
cohort               71
dtype: int64

In [21]:
#drop the one person with no cohort assigned
first_access_date= first_access_date.dropna()

In [22]:
first_access_date.cohort

1      [8.0, 28.0]
2           [22.0]
3           [22.0]
4           [22.0]
5           [22.0]
          ...     
906        [137.0]
907        [137.0]
908        [137.0]
909        [137.0]
910        [137.0]
Name: cohort, Length: 840, dtype: object

In [135]:
first_access_date

,id,first_access_date,cohort
1,2,2018-01-26,"[8.0, 28.0]"
2,3,2018-01-26,[22.0]
3,4,2018-01-26,[22.0]
4,5,2018-01-26,[22.0]
5,6,2018-01-26,[22.0]
...,...,...,...
976,977,2021-04-12,[139.0]
977,978,2021-04-12,[139.0]
978,979,2021-04-12,[139.0]
979,980,2021-04-13,[139.0]


In [23]:
id_by_first_access_date = first_access_date.groupby(['first_access_date']).count()

In [ ]:
df

In [47]:
df.isna().sum()

date          0
time          0
page          1
id            0
cohort    52893
ip            0
dtype: int64

In [140]:
#replace na values with unknown
df = df.fillna('UNKNOWN')

In [10]:
id_counts = df.groupby('id')[['page','date','cohort']].nunique()
id_counts

,page,date,cohort
id,,,
1,976,637,2
2,151,83,1
3,169,107,1
4,143,54,1
5,197,82,1
...,...,...,...
977,12,10,1
978,18,9,1
979,20,9,1


In [11]:
cohort_counts = df.groupby('cohort')[['page','date','id']].nunique()
cohort_counts

,page,date,id
cohort,,,
1.0,342,278,18
2.0,44,12,3
4.0,4,1,1
5.0,1,1,1
6.0,45,12,2
7.0,143,46,6
8.0,278,162,5
9.0,4,1,1
11.0,95,18,5


# Which lesson appears to attract the most traffic consistently across cohorts (per program)?

In [30]:
# Group by 'program' and 'page', and count the number of visits per page
page_visits_by_program = df.groupby(['program', 'page']).size().reset_index(name='visits')

# Find the page with the most visits for each program
most_visited_pages = page_visits_by_program.loc[page_visits_by_program.groupby('program')['visits'].idxmax()]

print(most_visited_pages[['program', 'page', 'visits']])

              program              page  visits
3        Data Science                 /    8358
683         Front End  content/html-css       2
690   Full Stack Java                 /   35814
2599   Full Stack PHP                 /    1681


In [35]:
# Filter rows where the 'page' column contains 'content'
df_filtered = df[df['page'].str.contains('content')]

In [36]:
# Group by 'program' and 'page', and count the number of visits per page
page_visits_by_program = df_filtered.groupby(['program', 'page']).size().reset_index(name='visits')

# Find the page with the most visits for each program
most_visited_pages = page_visits_by_program.loc[page_visits_by_program.groupby('program')['visits'].idxmax()]

print(most_visited_pages[['program', 'page', 'visits']])

             program              page  visits
0          Front End  content/html-css       2
154  Full Stack Java     content/php_i     162
339   Full Stack PHP  content/html-css     262


In [37]:
# Filter rows where the 'program' column is 'Data Science'
df_data_science = df[df['program'] == 'Data Science']

# Group by 'page' and count the number of visits per page
page_visits = df_data_science.groupby('page').size().reset_index(name='visits')

# Sort the DataFrame by the 'visits' column in descending order
most_viewed_pages = page_visits.sort_values('visits', ascending=False)

# Display the most viewed pages for the 'Data Science' program
print(most_viewed_pages)

                                         page  visits
3                                           /    8358
592                  search/search_index.json    2203
397                   classification/overview    1785
19   1-fundamentals/modern-data-scientist.jpg    1655
17       1-fundamentals/AI-ML-DL-timeline.jpg    1651
..                                        ...     ...
407           cli/4-navigating-the-filesystem       1
583                   regression/project/null       1
426                           creating-charts       1
429               curie-statistics-assessment       1
0             %20https://github.com/RaulCPena       1

[682 rows x 2 columns]


|    # |            program |                       page | visits |
|-----:|-------------------:|--------------------------:|-------:|
|    0 |         Front End  |           content/html-css |      2 |
|  154 |  Full Stack Java  |             content/php_i  |    162 |
|  339 |   Full Stack PHP  |           content/html-css |    262 |
|  397 |     Data Science  |  classification/overview   |   1785 |


# Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?

In [42]:
# Filter out rows where the 'page' column doesn't match the specified conditions
df_filtered = df[~df['page'].str.contains(r'^(\/|toc|.*\.json)$')]

In [42]:
# Group by 'cohort_name' and 'page', and count the number of visits per page
page_visits_by_cohort = df_filtered.groupby(['cohort_name', 'page']).size().reset_index(name='visits')

# Find the top visited pages for each cohort
most_visited_pages = page_visits_by_cohort.loc[page_visits_by_cohort.groupby('cohort_name')['visits'].idxmax()]

# Sort the DataFrame by the 'visits' column in descending order
most_visited_pages_sorted = most_visited_pages.sort_values('visits', ascending=False)

# Display the most visited pages for each cohort
print(most_visited_pages_sorted)

# Calculate the mean and standard deviation of the visits
mean_visits = most_visited_pages_sorted['visits'].mean()
std_visits = most_visited_pages_sorted['visits'].std()

# Filter cohorts with significantly more visits (more than 1 standard deviation above the mean)
significant_cohorts = most_visited_pages_sorted[most_visited_pages_sorted['visits'] > mean_visits + std_visits]

# Display the cohorts with significantly more visits to a specific page
print("\nCohorts with significantly more visits to a specific page:")
print(significant_cohorts)

C:\Users\juang\AppData\Local\Temp\ipykernel_5112\2127629163.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_filtered = df[~df['page'].str.contains(r'^(\/|toc|.*\.json)$')]


      cohort_name                                               page  visits
10449       Staff                                       javascript-i    1817
3063       Darden                            classification/overview    1109
2322        Ceres                                       javascript-i    1003
5823      Jupiter                                       javascript-i     926
5330     Hyperion                                       javascript-i     910
7639        Marco                                       javascript-i     907
13231        Zion                                       javascript-i     897
3412       Deimos                                       javascript-i     896
11913   Voyageurs                                       javascript-i     884
6886       Lassen                                         index.html     877
4326      Fortuna                                           java-iii     786
4650     Ganymede                                       javascript-i     760

# Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?

In [43]:

# Filter rows where the 'date' column is between 'start_date' and 'end_date'
df_active = df[(df['date'] >= df['start_date']) & (df['date'] <= df['end_date'])]

# Group by 'id' and count the number of page visits
page_visits_by_id = df_active.groupby('id').size().reset_index(name='visits')

# Sort the DataFrame by the 'visits' column in ascending order
sorted_visits_by_id = page_visits_by_id.sort_values('visits', ascending=True)

# Define a threshold for "hardly accessing pages"
# For example, you can set it to a specific number of visits, like 5 or 10
threshold = 5

# Filter ids with visits less than or equal to the threshold
hardly_accessing_pages = sorted_visits_by_id[sorted_visits_by_id['visits'] <= threshold]

# Get the ids of these students
hardly_accessing_ids = hardly_accessing_pages['id'].tolist()

# Get the information about these students from the original DataFrame
students_info = df[df['id'].isin(hardly_accessing_ids)].drop_duplicates(subset='id')

# Display the information about these students
print(students_info)

                 date      time             ip page   id  cohort cohort_name   
date                                                                           
2018-09-27 2018-09-27  13:57:44   97.105.19.58    /  278    24.0   Voyageurs  \
2019-11-04 2019-11-04  16:19:26   97.105.19.58    /  539    52.0      Europa   
2020-03-23 2020-03-23  14:10:17   97.105.19.58    /  619    57.0    Ganymede   
2020-12-07 2020-12-07  14:22:00   69.154.52.98    /  832    62.0     Jupiter   
2021-01-26 2021-01-26  12:21:18  136.50.50.187    /  879   135.0       Marco   
2021-03-15 2021-03-15  16:59:56  71.221.46.112    /  918   138.0     Neptune   
2021-03-15 2021-03-15  17:00:37  70.121.129.79    /  940   138.0     Neptune   

                 slack start_date   end_date  program_id          program  
date                                                                       
2018-09-27  #voyageurs 2018-05-29 2018-10-11           2  Full Stack Java  
2019-11-04     #europa 2019-11-04 2020-04-17       

# Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?

To identify suspicious activity, you can look for patterns in the data that indicate unauthorized access or web-scraping. Here are some things to consider:

Cross-program access: Check if there are any ids accessing pages associated with a program that they are not enrolled in. This may indicate unauthorized access or sharing of login credentials.

Excessive page access: Check if there are any ids that are accessing an unusually high number of pages in a short period of time. This may indicate web-scraping or automated access.

Unusual IP addresses: Check if there are any IP addresses that are accessing an unusually high number of pages, or that are associated with a large number of ids. This may indicate bot traffic or web-scraping.

Suspicious paths or query strings: Check if there are any unusual paths or query strings in the page URLs. This may indicate web-scraping or automated access.

User agents: Check the user agents associated with the page accesses. If there are a large number of requests coming from a single user agent, or if the user agent is associated with a known web-scraping tool, this may indicate web-scraping.

To investigate these potential issues, you can use a combination of data filtering, aggregation, and visualization techniques. For example, you can filter the data to focus on a specific time period or program, and then use aggregation functions to identify patterns in the data. You can also use visualization tools to identify outliers or unusual patterns in the data. Additionally, you may want to investigate individual ids or IP addresses that appear to be associated with suspicious activity.

It's important to note that some of these patterns may have legitimate explanations, so it's important to investigate further before making any conclusions.

In [63]:
# Convert the 'time' column to a datetime object
df['time'] = pd.to_datetime(df['time'])

# Define the time window over which to check for unusually high page accesses
time_window = 5

# Group the DataFrame by user ID and page, and count the number of accesses within each time window
counts = df.groupby(['id', 'page'])['time'].rolling(f'{time_window}min').count()

# Filter the counts DataFrame to only include counts that are above a certain threshold
threshold = 10
unusual_counts = counts[counts > threshold]

# Get the IDs associated with the unusually high page counts
unusual_ids = unusual_counts.index.get_level_values('id').unique()

# Print the list of IDs with unusually high page counts
print('IDs with unusually high page counts:')
print(unusual_ids)

C:\Users\juang\AppData\Local\Temp\ipykernel_5112\3948565881.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['time'] = pd.to_datetime(df['time'])


ValueError: window must be an integer 0 or greater

# At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?

In [55]:
# Reset the index of the DataFrame
df = df.reset_index(drop=True)

# Filter rows where the 'date' column is in 2019
df_2019 = df[df['date'].dt.year == 2019]

# Define a function to check if ids access pages from other programs
def is_cross_program(row):
    web_dev_programs = ['Full Stack Java', 'Full Stack PHP', 'Front End']
    data_science_program = 'Data Science'
    
    if row['program'] in web_dev_programs and row['page'].startswith('content/data_science/'):
        return True
    elif row['program'] == data_science_program and row['page'].startswith('content/web_dev/'):
        return True
    else:
        return False

# Apply the function to the DataFrame
df_2019['cross_program'] = df_2019.apply(is_cross_program, axis=1)

# Group by month and count the number of cross-program accesses
cross_program_access_by_month = df_2019[df_2019['cross_program']].groupby(df_2019['date'].dt.to_period('M')).size().reset_index(name='count')

# Display the cross-program access count by month
print(cross_program_access_by_month)

# Find the first index with zero cross-program accesses
first_index_no_cross_access = cross_program_access_by_month[cross_program_access_by_month['count'] == 0].first_valid_index()

# Check if the index exists and display the corresponding month
if first_index_no_cross_access is not None:
    first_month_no_cross_access = cross_program_access_by_month.loc[first_index_no_cross_access, 'date']
    print(f"\nThe first month with zero cross-program accesses: {first_month_no_cross_access}")
else:
    print("\nThere is no month with zero cross-program accesses.")

Empty DataFrame
Columns: [date, count]
Index: []

There is no month with zero cross-program accesses.


C:\Users\juang\AppData\Local\Temp\ipykernel_5112\976919691.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2019['cross_program'] = df_2019.apply(is_cross_program, axis=1)


In [58]:
# Reset the index of the DataFrame
df = df.reset_index(drop=True)

# Filter rows where the 'date' column is in the last quarter of 2019
last_quarter_2019 = pd.date_range(start='2019-10-01', end='2019-12-31')
df_last_quarter_2019 = df[df['date'].isin(last_quarter_2019)]

# Remove users with cross-program access in the last quarter of 2019
web_dev_programs = ['Full Stack Java', 'Full Stack PHP', 'Front End']
data_science_program = 'Data Science'
df_last_quarter_2019 = df_last_quarter_2019[~((df_last_quarter_2019['program'].isin(web_dev_programs) & df_last_quarter_2019['page'].str.startswith('content/data_science/')) | (df_last_quarter_2019['program'] == data_science_program & df_last_quarter_2019['page'].str.startswith('content/web_dev/')))]

# Filter rows where the 'date' column is in 2019
df_2019 = df[df['date'].dt.year == 2019]

# Define a function to check if ids access pages from other programs
def is_cross_program(row):
    web_dev_programs = ['Full Stack Java', 'Full Stack PHP', 'Front End']
    data_science_program = 'Data Science'
    
    if row['program'] in web_dev_programs and row['page'].startswith('content/data_science/'):
        return True
    elif row['program'] == data_science_program and row['page'].startswith('content/web_dev/'):
        return True
    else:
        return False

# Apply the function to the DataFrame
df_2019['cross_program'] = df_2019.apply(is_cross_program, axis=1)

# Group by month and count the number of cross-program accesses
cross_program_access_by_month = df_2019[df_2019['cross_program']].groupby(df_2019['date'].dt.to_period('M')).size().reset_index(name='count')

# Display the cross-program access count by month
print(cross_program_access_by_month)

# Find the first index with zero cross-program accesses
first_index_no_cross_access = cross_program_access_by_month[cross_program_access_by_month['count'] == 0].first_valid_index()

# Check if the index exists and display the corresponding month
if first_index_no_cross_access is not None:
    first_month_no_cross_access = cross_program_access_by_month.loc[first_index_no_cross_access, 'date']
    print(f"\nThe first month with zero cross-program accesses: {first_month_no_cross_access}")
else:
    print("\nThere is no month with zero cross-program accesses.")

TypeError: Cannot perform 'rand_' with a dtyped [bool] array and scalar of type [bool]

In [59]:
def get_random_ids(df, cohort):
    """
    Returns a list of random user ids for the specified cohort from the DataFrame.
    """
    cohort_df = df[df['cohort'] == cohort]
    user_ids = cohort_df['id'].unique()
    active_user_ids = cohort_df[(cohort_df['start_date'] <= cohort_df['date']) & (cohort_df['date'] <= cohort_df['end_date'])]['id'].unique()
    inactive_user_ids = np.setdiff1d(user_ids, active_user_ids)
    if len(inactive_user_ids) > 0:
        random_inactive_user_ids = np.random.choice(inactive_user_ids, size=min(len(inactive_user_ids), 5), replace=False)
        return random_inactive_user_ids.tolist()
    else:
        return []

# What topics are grads continuing to reference after graduation and into their jobs (for each program)?

In [51]:
# Filter rows where the 'date' column is greater than 'end_date'
df_post_end_date = df_filtered[df_filtered['date'] > df_filtered['end_date']]

# Group by 'program' and 'page', and count the number of visits per page
page_visits_by_program = df_post_end_date.groupby(['program', 'page']).size().reset_index(name='visits')

# Find the top visited pages for each program after an id has passed their end_date
most_visited_pages = page_visits_by_program.loc[page_visits_by_program.groupby('program')['visits'].idxmax()]

# Sort the DataFrame by the 'visits' column in descending order
most_visited_pages_sorted = most_visited_pages.sort_values('visits', ascending=False)

# Display the most visited pages after an id has passed their end_date by program
print(most_visited_pages_sorted[['program', 'page', 'visits']])

              program                page  visits
1768  Full Stack Java        javascript-i    4229
2588   Full Stack PHP          index.html    1011
386      Data Science  sql/mysql-overview     275
435         Front End    content/html-css       2


In [52]:
# Filter rows where the 'date' column is greater than the 'end_date'
df_post_end_date = df_filtered[df_filtered['date'] > df_filtered['end_date']]

# Group by 'page' and 'program', and count the number of visits
page_visits_by_program = df_post_end_date.groupby(['program', 'page']).size().reset_index(name='visits')

# Sort the DataFrame by the 'visits' column in descending order
sorted_page_visits_by_program = page_visits_by_program.sort_values('visits', ascending=False)

# Get the top 5 pages with the most views after the end_date for each program
top_pages_by_program = sorted_page_visits_by_program.groupby('program').head(5)

# Display the top 5 pages with the most views after the end_date for each program
print(top_pages_by_program)

              program                                            page  visits
1768  Full Stack Java                                    javascript-i    4229
1976  Full Stack Java                                          spring    3760
1699  Full Stack Java                                        html-css    3136
1754  Full Stack Java                                        java-iii    3058
1742  Full Stack Java                                         java-ii    2985
2588   Full Stack PHP                                      index.html    1011
2626   Full Stack PHP                                    javascript-i     736
2565   Full Stack PHP                                        html-css     542
2756   Full Stack PHP                                          spring     501
2615   Full Stack PHP                                        java-iii     479
386      Data Science                              sql/mysql-overview     275
251      Data Science                         classification/ove

# Which lessons are least accessed?

In [48]:

# Group by 'page' and count the number of visits
page_visits = df.groupby('page').size().reset_index(name='visits')

# Sort the DataFrame by the 'visits' column in ascending order
least_visited_pages = page_visits.sort_values('visits', ascending=True)

# Filter pages with visits greater than or equal to 15
filtered_least_visited_pages = least_visited_pages[least_visited_pages['visits'] >= 15]

# Display the filtered least visited pages
print(filtered_least_visited_pages)

                                                   page  visits
530                                         6.3_Acquire      15
952                               appendix/testing-code      15
1102         content/appendix/postwork/basic_challenges      15
843   appendix/extra-challenges/mysql/mysql-extra-ex...      15
1222          content/jquery/ajax/ajax-api-request.html      15
...                                                 ...     ...
1685                                           java-iii   13166
2007                           search/search_index.json   17534
2190                                                toc   17591
1712                                       javascript-i   18203
6                                                     /   45854

[957 rows x 2 columns]


In [49]:
# Group by 'page' and count the number of visits
page_visits = df.groupby('page').size().reset_index(name='visits')

# Calculate the average cohort size by program
avg_cohort_size_by_program = df.groupby('program')['cohort'].nunique().reset_index(name='avg_cohort_size')

# Merge the page_visits DataFrame with the average cohort size by program
page_visits_with_avg_cohort = page_visits.merge(df[['page', 'program']].drop_duplicates(), on='page', how='left')
page_visits_with_avg_cohort = page_visits_with_avg_cohort.merge(avg_cohort_size_by_program, on='program', how='left')

# Drop pages with visits lower than the average cohort size by program
filtered_page_visits = page_visits_with_avg_cohort[page_visits_with_avg_cohort['visits'] >= page_visits_with_avg_cohort['avg_cohort_size']]

# Sort the DataFrame by the 'visits' column in ascending order
least_accessed_pages = filtered_page_visits.sort_values('visits', ascending=True)

# Display the least accessed pages with visits greater than or equal to the average cohort size by program
print(least_accessed_pages[['program', 'page', 'visits']])

              program                                 page  visits
2799     Data Science                      nlp/pos-tagging       5
2830     Data Science                              prepare       5
2340     Data Science          grades/getUserDetails/916/3       5
808      Data Science            7-classification/ensemble       5
1483     Data Science  clustering/I_have_clusters_now_what       5
...               ...                                  ...     ...
2544     Data Science                         javascript-i   18203
10       Data Science                                    /   45854
9           Front End                                    /   45854
8     Full Stack Java                                    /   45854
7      Full Stack PHP                                    /   45854

[1772 rows x 3 columns]


In [11]:
# Filter out pages containing 'git'
df = df[~df['page'].str.contains('git')]

# Group by 'page' and 'program', count the number of visits
page_visits = df.groupby(['program', 'page']).size().reset_index(name='visits')

# Calculate the mean and standard deviation of visits by program
stats = page_visits.groupby('program')['visits'].agg(['mean', 'std'])

# Merge stats with page visits
page_visits = page_visits.merge(stats, on='program', how='left')

# Compute the lower limit for two standard deviations of page visits
page_visits['lower_limit'] = page_visits['mean'] - 2 * page_visits['std']

# Filter the DataFrame to include only the rows within two standard deviations of the mean
page_visits = page_visits[page_visits['visits'] >= page_visits['lower_limit']]

# Sort the DataFrame by 'program' and 'visits' column in ascending order
sorted_page_visits = page_visits.sort_values(['program', 'visits'], ascending=True)

# Group by 'program' and pick the least 5 visited pages
least_visited_pages_by_program = sorted_page_visits.groupby('program').head(5)

# Display the least visited pages per program
print(least_visited_pages_by_program[['program', 'page', 'visits']])


              program                                               page   
50       Data Science                12-distributed-ml/3-getting-started  \
52       Data Science                        12-distributed-ml/4-acquire   
55       Data Science                        12-distributed-ml/5-inspect   
57       Data Science               12-distributed-ml/6.1-prepare-part-1   
58       Data Science               12-distributed-ml/6.2-prepare-part-2   
667         Front End                                                  /   
669         Front End                 content/html-css/introduction.html   
668         Front End                                   content/html-css   
670   Full Stack Java                                                  '   
673   Full Stack Java                                                00_   
703   Full Stack Java                         1._Fundamentals/index.html   
706   Full Stack Java  10-anomaly-detection/2-continuous-probabilisti...   
708   Full S

# Anything else I should be aware of?